# CI/CD in Generative AI Development Series
## LLM Stability Testing with Python Unittests 

This notebook showcases how to run Stability Tests using unit testing frameworks available in python. Stability testing is important in Generative AI applications especially the ones using LLM APIs like OpenAI, Anthropic, Gemini, BedRock etc.,. The LLMs behind the APIs are frequently updated and hot swapped leading to change in performance of the LLM. Consumers of these APIs should have automated stability testing frameworks to intercept any drift in LLM output. Stability testing for Generative AI acts as the canary in the coal mine for these consumer applications.

## Scenario

An process automation application that uses an LLM to ingest bills from users and scrapes data from them to upload to a backend office application. The stability of the GPT-4o endpoint in this scenario is its ability to extract correct information from the bills over time. To make sure that the performance of the LLM is not falling, developers will need to write tests that measure the performance of the LLM over an evaluation dataset and provide alerts if the performance tests fail. This is quite similar to unit testing in CI/CD for software development. 

### Input Data

For this showcase, we are using the kaggle dataset provided by the use ------. To test the stability of our LLM we have created an evaluation dataset that has a input and expected output and can be used to test if the LLM is stable or not.

In [62]:
llm_test_data = pd.DataFrame({'filename':['chowderhut_20231005_011.pdf','shell_20231005_003.pdf',
                                          'beerhouse_20231209_005.pdf','dennys_20231209_004.pdf',
                                          'cafemason_20231005_009.pdf','topgolf_20231209_011.pdf',
                                          'yellow_20231209_008.pdf'],
                              'output':[{'category': 'food', 'total_bill_amount': 21.15},
                                        {'category': 'transport', 'total_bill_amount': 28.32},
                                        {'category': 'food', 'total_bill_amount': 41.72},
                                        {'category': 'food', 'total_bill_amount': 58.44},
                                        {'category': 'food', 'total_bill_amount': 32.59},
                                        {'category': 'other', 'total_bill_amount': 155.68},
                                        {'category': 'transport', 'total_bill_amount': 43.02},
                                       ]
                             })
llm_test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  7 non-null      object
 1   output    7 non-null      object
dtypes: object(2)
memory usage: 244.0+ bytes


In [63]:
llm_test_data.head()

,filename,output
0,chowderhut_20231005_011.pdf,"{'category': 'food', 'total_bill_amount': 21.15}"
1,shell_20231005_003.pdf,"{'category': 'transport', 'total_bill_amount':..."
2,beerhouse_20231209_005.pdf,"{'category': 'food', 'total_bill_amount': 41.72}"
3,dennys_20231209_004.pdf,"{'category': 'food', 'total_bill_amount': 58.44}"
4,cafemason_20231005_009.pdf,"{'category': 'food', 'total_bill_amount': 32.59}"


### LLM Function
The LLM completion function to be tested. This function simulates the endpoint which will ingest text from a bill to be reimbursed and return the bill amount and the category of the bill. The function uses GPT-4o model as the LLM but this notebook can be used with any LLM API or self hosted LLM as well.

In [8]:
import json
import pymupdf4llm
import pandas as pd
from openai import OpenAI

key = json.load(open('./openai_key.json'))['key']

def prompt_llm(bill_text, key):
    client = OpenAI(api_key=key)
    completion = client.chat.completions.create(
    model="gpt-4o",
      messages=[
        {"role": "system", "content": "You are an AI assistant, skilled in helping \
         extract billing information from invoices and bills to help in reimbursement processes."},
        {"role": "user", "content": "Using the following extracted information from a bill in triple quotes, generate \
          total_bill_amount, category which is one of food, transport, other as output. \
         Output only in json format as {\"total_bill_amount\":, \"category\":}. \
         \n Bill Extract: \n \n \n '''" + bill_text + "'''"}
      ],
      seed = 42
    )
    return completion.choices[0].message.content

### Stability Tests using Python Unittest

Python's [unittest library](https://docs.python.org/3/library/unittest.html) is the defacto unit testing library that is included in the python distribution. Unittest recommends a class based approach to unit testing functions. All the tests to be executed need to be wrapped inside the [TestCase class](https://docs.python.org/3/library/unittest.html#unittest.TestCase) and should start with "test" in the name.

In our scenario, we will include three test cases with varying levels of stability checks on the LLM API. The tests will be executed in the required pipelines to ensure that the LLM is stable and no interventions are required. 

In [59]:
import unittest

class LLMStability(unittest.TestCase):
        
    def test_one_row(self):
        # Testing a single row
        row = llm_test_data.iloc[0]
        md_text = pymupdf4llm.to_markdown("./bills/"+row['filename'])
        response = prompt_llm(md_text, key)
        response_json = json.loads(response[response.find('{'):response.rfind('}')+1])
        self.assertDictEqual(response_json, {'category': 'food', 'total_bill_amount': 21.15})

    def test_llm_output_sample(self):
        # Testing a sample of rows
        for i,row in llm_test_data.head(3).iterrows():
            md_text = pymupdf4llm.to_markdown("./bills/"+row['filename'])
            response = prompt_llm(md_text, key)
            response_json = json.loads(response[response.find('{'):response.rfind('}')+1])
            self.assertDictEqual(response_json, row['output'])
    
    def test_llm_output_full(self):
        ROWS_CORRECT = 0
        EXPECTED_MATCH_RATE = 0.5
        for i,row in llm_test_data.iterrows():
            # Testing entire evaluation dataset and checking threshold of at least 50% match rate
            md_text = pymupdf4llm.to_markdown("./bills/"+row['filename'])
            response = prompt_llm(md_text, key)
            response_json = json.loads(response[response.find('{'):response.rfind('}')+1])
            if(response_json == row['output']):
                ROWS_CORRECT=ROWS_CORRECT+1
        self.assertGreaterEqual(ROWS_CORRECT/llm_test_data.shape[0],EXPECTED_MATCH_RATE)

In [60]:
result = unittest.main(argv=[''], verbosity=1, exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 10.343s

OK


In [61]:
print('Test Results \n'+''.join(['-' for i in range(30)])+'\nTotal Tests Executed: {}\nTests Failed: {}\nTest Execution Run Success: {}'.format(
    result.result.testsRun,
    len(result.result.failures),
    result.result.wasSuccessful()))

Test Results 
------------------------------
Total Tests Executed: 3
Tests Failed: 0
Test Execution Run Success: True


In [14]:
import ipytest
ipytest.autoconfig()

In [15]:
#!pip install ipytest

In [39]:
def test_my_stuff():
    assert 42 == 42

In [40]:
results = ipytest.run()

....                                                                                         [100%]
========================================= warnings summary =========================================
../../../../../../opt/anaconda3/envs/devenv311/lib/python3.11/site-packages/_pytest/config/__init__.py:1277
  /opt/anaconda3/envs/devenv311/lib/python3.11/site-packages/_pytest/config/__init__.py:1277: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: anyio
    self._mark_plugins_for_rewrite(hook)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
4 passed, 1 warning in 13.11s


In [41]:
results.TESTS_FAILED.value

1

In [42]:
results.NO_TESTS_COLLECTED.value

5

In [43]:
results.OK.value

0